# Classes com métodos de análise e previsão de séries

## Classe pré-processamento

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

class PreProcessamento():
    '''
    Classe com métodos de previsão para séries temporais utilizando poucas variáveis
    globais para dar maior flexibilidade às chamadas externas dos métodos. 
    
    Alerta: Métodos analisam apenas uma coluna(variável) do timeseries por vez.
    # coluna_analise
    '''
    
    def __init__(self,df): 
        '''
        Método init. Recebe o dataframe(df) pré-processado para aumentar flexibilidade nas chamadas de instâncias 
        diversas.
        '''
        try: self.df = df            
        except:
            print('Erro de entrada! Dataframe deve ser informado pré-formatado com pd.read_csv()')

    def periodo(self):
        '''
        Método separa o df em frequência diária, semanal e mensal para análise posterior 
        realizando a soma dos períodos por default.
        Informe fun=mean(),max(),min() conforme desejado.
        '''
        dia = self.df.resample('D').sum()
        semana = self.df.resample('W').sum()
        mes = self.df.resample('M').sum()
        return dia,semana,mes

    def forecast(self,df):
        ''' Método identifica a frequencia do df e gera incremento de um período futuro '''
        
        freq = df.index.freq              
        return pd.date_range(df.index[-1], periods=2, freq = freq)[1] 

    def div_train_test(self,df):
        '''Método realiza a separação do conjunto de dados para teste e treinamentos'''        
        corte = int(len(df)*2/3) 
        train_set = df[:corte]
        test_set  = df[corte:]
        return train_set,test_set

    def graf(self, metodo, train_set,test_set, previsoes, erro=float, n=None, forecast=None):
        '''Método gera gráficos para datasérie diferentes
        Utiliza train_set,test_set, previsoes
        '''

        plt.plot(train_set)
        plt.plot(test_set)
        plt.plot(previsoes)
        plt.plot(forecast,'or')
        plt.title(f'Método:{metodo}, erro (RMSE): {erro}, previsto:{round(forecast[0],2)}')
        plt.legend(['Dados_treino','Dados_teste','Resultado_test','Previsão'])  



## Classe Métodos de previsão


In [23]:
class MetodosPrevisao(PreProcessamento): 
    '''Classe com os métodos estatísticos de previsão de séries temporais:
    AR, ARMA, ARIMA e SARIMA.
    '''
    def __init__(self,df):
        
        ''' Classe herda atributos da classe Preprocessamento.'''
        PreProcessamento.__init__(self,df)

        
    def ar(self, df, lags, graf = 0):
        '''Modelo AutoRegressivo e teste e 'n' configurações.
        * Chamada interna do train_set e test_set.
        * Previsão para período seguinte
        * Saída gráfica dos dados: train, test e previsoes se 'graf=1'
        '''
        train_set,test_set = self.div_train_test(df)
        ar = AutoReg(train_set, lags = lags).fit()

        # Recebe os parâmetros do modelo treinado e aplica no conjunto de teste
        previsoes = ar.params[0]
        for i,coef in enumerate(ar.params[1:]):
            previsoes+= coef * test_set.shift(i+1)
            erro = round((((test_set - previsoes)**2).mean())**0.5,2)
        
        forecast = ar.predict(start=len(df), end=len(df))
        
        if graf==1:
            self.graf('AutoRegressivo',train_set, test_set, previsoes, erro,lags,forecast)
        else: 
            None  

        return erro, previsoes,forecast 

    
    def arma_arima(self, df, order=(2, 0, 0), graf = 0):
        '''Modelo(ARMA) Auto Regressivo de Médias Móveis (Autoregressive Moving Average) e teste e 'n' configurações.
        * Chamada interna do train_set e test_set.
        * Previsão para período seguinte
        * Saída gráfica dos dados: train, test e previsoes se 'graf=1'
        * arma(p,q) é utilizado a partir de ARIMA mantendo zero o integrador d
        ARIMA order = (p,0,q)
        '''
        p,d,q = order[0],order[1],order[2]
        model=None
        if (p==0 and d==0 and q!=0): model='MA'
        if (p!=0 and d==0 and q==0): model='AR'
        if (p!=0 and d==0 and q!=0): model='ARMA'
        if (p!=0 and d!=0 and q!=0): model='ARIMA'
        
        train_set, test_set = self.div_train_test(df)
        arma_train = ARIMA(train_set, order = order).fit()
        arma_test = ARIMA(test_set, order = order).fit(arma_train.params)
        previsoes = arma_test.predict()
        forecast = arma_test.forecast()
        erro = round((((test_set - previsoes)**2).mean())**0.5,2)

        lags=order
        if graf==1:
            self.graf(f'{model}',train_set, test_set, previsoes, erro,lags,forecast)
        else: 
            None  

        print(f'Modelo: {model}, erro(RMSE): {erro}\nforecast: {forecast}.\n')  
    
    def sarima(self, df, order=(1, 0, 0),s_order=(0, 0, 0, 0), graf = 0):
        '''Modelo(ARMA) Auto Regressivo de Médias Móveis (Autoregressive Moving Average) e teste e 'n' configurações.
        * Chamada interna do train_set e test_set.
        * Previsão para período seguinte
        * Saída gráfica dos dados: train, test e previsoes se 'graf=1'
        * Modelos:
            - modelos autorregressivos: AR (p)
            - modelos de média móvel: MA (q)
            - modelos de média móvel autorregressiva mista: ARMA (p, q)
            - modelos de integração: ARIMA (p, d, q)
            - modelos sazonais: SARIMA (P, D, Q, s)
            
        '''
       
        if (P!=0 or D!=0 or Q!=0): model='SARIMA'
        
        train_set, test_set = self.div_train_test(df)
        sarima_train = SARIMAX(train_set, order = order, seasonal_order = order_s).fit()
        sarima_test = SARIMAX(test_set, order = order, seasonal_order = order_s).fit(arma_train.params)
        previsoes = arma_test.predict()
        forecast = arma_test.forecast()
        erro = round((((test_set - previsoes)**2).mean())**0.5,2)

        lags=order
        if graf==1:
            self.graf(f'{model}',train_set, test_set, previsoes, erro,lags,forecast)
        else: 
            None  

        print(f'Modelo: {model}, erro(RMSE): {erro}\nforecast: {forecast}.\n')  
    
    
    
    def analise_grafica(self,lista,order,col=0):
        '''Análise gráfico dos modelos AR, MO, ARMA e ARIMA recebe:
        * uma lista de df com períodos distintos [dia,sem,mes]
        * order dos componentes (p,d,q)
        * coluna de de análise do df, cause tenha mais de uma.
        '''
        
        plt.figure(figsize=(25,6))
        for i,j in enumerate(lista):   
            plt.subplot(1,len(lista),i+1)
            self.arma_arima(j[j.columns[col]],order,1)   # Utilizando 3 lag 
            

        print(f"Gráficos de análise da coluna '{j.columns[col]}' do dataset\n")
        